In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import ShuffleSplit
from sklearn import metrics
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('/content/drive/MyDrive/My BSSE/My 8th Semester/Software Maintenance/1006_1015_1027_maintenance_assignment_1/commits_dataset.csv', sep='#')
print(dataset.shape)

(1151, 72)


In [4]:
dataset.to_excel("output.xlsx")

In [5]:
 list(dataset.columns.values)

['commitId',
 'project',
 'comment',
 'label',
 'ADDING_ATTRIBUTE_MODIFIABILITY',
 'ADDING_CLASS_DERIVABILITY',
 'ADDING_METHOD_OVERRIDABILITY',
 'ADDITIONAL_CLASS',
 'ADDITIONAL_FUNCTIONALITY',
 'ADDITIONAL_OBJECT_STATE',
 'ALTERNATIVE_PART_DELETE',
 'ALTERNATIVE_PART_INSERT',
 'ATTRIBUTE_RENAMING',
 'ATTRIBUTE_TYPE_CHANGE',
 'CLASS_RENAMING',
 'COMMENT_DELETE',
 'COMMENT_INSERT',
 'COMMENT_MOVE',
 'COMMENT_UPDATE',
 'CONDITION_EXPRESSION_CHANGE',
 'DECREASING_ACCESSIBILITY_CHANGE',
 'DOC_DELETE',
 'DOC_INSERT',
 'DOC_UPDATE',
 'INCREASING_ACCESSIBILITY_CHANGE',
 'METHOD_RENAMING',
 'PARAMETER_DELETE',
 'PARAMETER_INSERT',
 'PARAMETER_ORDERING_CHANGE',
 'PARAMETER_RENAMING',
 'PARAMETER_TYPE_CHANGE',
 'PARENT_CLASS_CHANGE',
 'PARENT_CLASS_DELETE',
 'PARENT_CLASS_INSERT',
 'PARENT_INTERFACE_CHANGE',
 'PARENT_INTERFACE_DELETE',
 'PARENT_INTERFACE_INSERT',
 'REMOVED_CLASS',
 'REMOVED_FUNCTIONALITY',
 'REMOVED_OBJECT_STATE',
 'REMOVING_ATTRIBUTE_MODIFIABILITY',
 'REMOVING_CLASS_DERIVABILI

In [6]:
def createHasKeywordLabel (keyword_features):
  hasKeyword = []
  hasKeywordInRow = 0
  for i in range(len(keyword_features)):
    hasKeywordInRow = 0
    keyword_row = keyword_features.loc[i]
    for j in range(len(keyword_row)):
      if keyword_row[j] != 0:
        hasKeyword.append(1)
        hasKeywordInRow = 1
        break

    if hasKeywordInRow == 0:
      hasKeyword.append(0)

  return hasKeyword

In [7]:
features = dataset.drop(['commitId','project','comment','label'], axis=1)
keyword_features = dataset.iloc[:, 52:72]
change_features = dataset.iloc[:, 4:52]
hasKeyword = createHasKeywordLabel(keyword_features)
features['hasKeyword'] = hasKeyword
keyword_features['hasKeyword'] = hasKeyword
change_features['hasKeyword'] = hasKeyword
dataset['hasKeyword'] = hasKeyword
print(dataset.columns.values)
labels = dataset[['label', 'hasKeyword']]

['commitId' 'project' 'comment' 'label' 'ADDING_ATTRIBUTE_MODIFIABILITY'
 'ADDING_CLASS_DERIVABILITY' 'ADDING_METHOD_OVERRIDABILITY'
 'ADDITIONAL_CLASS' 'ADDITIONAL_FUNCTIONALITY' 'ADDITIONAL_OBJECT_STATE'
 'ALTERNATIVE_PART_DELETE' 'ALTERNATIVE_PART_INSERT' 'ATTRIBUTE_RENAMING'
 'ATTRIBUTE_TYPE_CHANGE' 'CLASS_RENAMING' 'COMMENT_DELETE'
 'COMMENT_INSERT' 'COMMENT_MOVE' 'COMMENT_UPDATE'
 'CONDITION_EXPRESSION_CHANGE' 'DECREASING_ACCESSIBILITY_CHANGE'
 'DOC_DELETE' 'DOC_INSERT' 'DOC_UPDATE' 'INCREASING_ACCESSIBILITY_CHANGE'
 'METHOD_RENAMING' 'PARAMETER_DELETE' 'PARAMETER_INSERT'
 'PARAMETER_ORDERING_CHANGE' 'PARAMETER_RENAMING' 'PARAMETER_TYPE_CHANGE'
 'PARENT_CLASS_CHANGE' 'PARENT_CLASS_DELETE' 'PARENT_CLASS_INSERT'
 'PARENT_INTERFACE_CHANGE' 'PARENT_INTERFACE_DELETE'
 'PARENT_INTERFACE_INSERT' 'REMOVED_CLASS' 'REMOVED_FUNCTIONALITY'
 'REMOVED_OBJECT_STATE' 'REMOVING_ATTRIBUTE_MODIFIABILITY'
 'REMOVING_CLASS_DERIVABILITY' 'REMOVING_METHOD_OVERRIDABILITY'
 'RETURN_TYPE_CHANGE' 'RETURN_T

In [8]:
def divideDataOnHasKeyword (data):
  dataKey = data.loc[data['hasKeyword'] == 1]
  dataNotKey = data.loc[data['hasKeyword'] == 0]
  return dataKey, dataNotKey

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.15, random_state=44)

In [10]:
def modelling (kw, kw_):
  rf_model_kw = RandomForestClassifier()
  gbm_model_kw = GradientBoostingClassifier()
  dtc_model_kw = DecisionTreeClassifier()
  rf_model_kw_ = RandomForestClassifier()
  gbm_model_kw_ = GradientBoostingClassifier()
  dtc_model_kw_ = DecisionTreeClassifier()

  training_result_scores_rf = []
  training_result_scores_gbm = []
  training_result_scores_dtc = []
  training_kappa_rf = []
  training_kappa_gbm = []
  training_kappa_dtc = []

  y_pred_rf_kw = ''
  y_pred_gbm_kw = ''
  y_pred_dtc_kw = ''
  y_pred_rf_kw_ = ''
  y_pred_gbm_kw_ = ''
  y_pred_dtc_kw_ = ''

  for x in range(5):
    cv = ShuffleSplit(n_splits=10)
    for (train, test), i in zip(cv.split(X_train, y_train), range(10)):

      internal_X_train = X_train.iloc[train]
      internal_y_train = y_train.iloc[train]
      internal_X_test = X_train.iloc[test]
      internal_y_test = y_train.iloc[test]

      internal_X_test_keyword, internal_X_test_no_keyword = divideDataOnHasKeyword(internal_X_test)

      internal_X_train.drop('hasKeyword', axis=1)
      internal_X_test.drop('hasKeyword', axis=1)
      internal_X_test_keyword.drop('hasKeyword', axis=1)
      internal_X_test_no_keyword.drop('hasKeyword', axis=1)

      if kw == 'combined':
        if kw_ == 'combined':
          rf_model_kw.fit(internal_X_train, internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train, internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train, internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train, internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train, internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train, internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw.predict(internal_X_test_keyword)
          y_pred_gbm_kw = gbm_model_kw.predict(internal_X_test_keyword)
          y_pred_dtc_kw = dtc_model_kw.predict(internal_X_test_keyword)
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword)
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword)
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword)

        elif kw_ == 'keywords':
          rf_model_kw.fit(internal_X_train, internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train, internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train, internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw.predict(internal_X_test_keyword)
          y_pred_gbm_kw = gbm_model_kw.predict(internal_X_test_keyword)
          y_pred_dtc_kw = dtc_model_kw.predict(internal_X_test_keyword)
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])

        elif kw_ == 'changes':
          rf_model_kw.fit(internal_X_train, internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train, internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train, internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw.predict(internal_X_test_keyword)
          y_pred_gbm_kw = gbm_model_kw.predict(internal_X_test_keyword)
          y_pred_dtc_kw = dtc_model_kw.predict(internal_X_test_keyword)
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
      elif kw == 'keywords':
        if kw_ == 'combined':
          rf_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train, internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train, internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train, internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_gbm_kw = gbm_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_dtc_kw = dtc_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword)
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword)
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword)

        elif kw_ == 'keywords':
          rf_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_gbm_kw = gbm_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_dtc_kw = dtc_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])

        elif kw_ == 'changes':
          rf_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_gbm_kw = gbm_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_dtc_kw = dtc_model_kw.predict(internal_X_test_keyword.iloc[:, 48:68])
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
      elif kw == 'changes':
        if kw_ == 'combined':
          rf_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train, internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train, internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train, internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_gbm_kw = gbm_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_dtc_kw = dtc_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword)
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword)
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword)

        elif kw_ == 'keywords':
          rf_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train.iloc[:, 48:68], internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_gbm_kw = gbm_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_dtc_kw = dtc_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 48:68])

        elif kw_ == 'changes':
          rf_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          gbm_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          dtc_model_kw.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          rf_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          gbm_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])
          dtc_model_kw_.fit(internal_X_train.iloc[:, 0:48], internal_y_train['label'])

          y_pred_rf_kw = rf_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_gbm_kw = gbm_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_dtc_kw = dtc_model_kw_.predict(internal_X_test_keyword.iloc[:, 0:48])
          y_pred_rf_kw_ = rf_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
          y_pred_gbm_kw_ = gbm_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])
          y_pred_dtc_kw_ = dtc_model_kw_.predict(internal_X_test_no_keyword.iloc[:, 0:48])


      y_pred_rf = np.concatenate((y_pred_rf_kw,y_pred_rf_kw_), axis=0)
      y_pred_gbm = np.concatenate((y_pred_gbm_kw,y_pred_gbm_kw_), axis=0)
      y_pred_dtc = np.concatenate((y_pred_dtc_kw,y_pred_dtc_kw_),axis=0)

      y_test_keyword, y_test_no_keyword = divideDataOnHasKeyword(internal_y_test)

      internal_y_test.drop('hasKeyword', axis = 1)
      y_test_keyword.drop('hasKeyword', axis = 1)
      y_test_no_keyword.drop('hasKeyword', axis = 1)

      y_test_merged = np.concatenate((y_test_keyword,y_test_no_keyword), axis=0)

      y_test_merged = y_test_merged[:,0]

      training_kappa_rf.append(cohen_kappa_score(y_pred_rf,y_test_merged))
      training_kappa_gbm.append(cohen_kappa_score(y_pred_gbm,y_test_merged))
      training_kappa_dtc.append(cohen_kappa_score(y_pred_dtc,y_test_merged))

      training_result_scores_rf.append(metrics.accuracy_score(y_test_merged,y_pred_rf))
      training_result_scores_gbm.append(metrics.accuracy_score(y_test_merged,y_pred_gbm))
      training_result_scores_dtc.append(metrics.accuracy_score(y_test_merged,y_pred_dtc))

  print("Training Dataset Performance")
  print("")
  print("Random Forest")
  print("Kappa")
  print(sum(training_kappa_rf)/len(training_kappa_rf))
  print("Average Cross Validation score :{}".format(sum(training_result_scores_rf)/len(training_result_scores_rf)))
  print("")
  print("GBM")
  print("Kappa")
  print(sum(training_kappa_gbm)/len(training_kappa_gbm))
  print("Average Cross Validation score :{}".format(sum(training_result_scores_gbm)/len(training_result_scores_gbm)))
  print("")
  print("Decision Tree")
  print("Kappa")
  print(sum(training_kappa_dtc)/len(training_kappa_dtc))
  print("Average Cross Validation score :{}".format(sum(training_result_scores_dtc)/len(training_result_scores_dtc)))

  print("")

  rf_model_kw = RandomForestClassifier()
  gbm_model_kw = GradientBoostingClassifier()
  dtc_model_kw = DecisionTreeClassifier()
  rf_model_kw_ = RandomForestClassifier()
  gbm_model_kw_ = GradientBoostingClassifier()
  dtc_model_kw_ = DecisionTreeClassifier()

  training_result_scores_rf = []
  training_result_scores_gbm = []
  training_result_scores_dtc = []
  training_kappa_rf = []
  training_kappa_gbm = []
  training_kappa_dtc = []

  y_pred_rf_kw = ''
  y_pred_gbm_kw = ''
  y_pred_dtc_kw = ''
  y_pred_rf_kw_ = ''
  y_pred_gbm_kw_ = ''
  y_pred_dtc_kw_ = ''

  X_test_keyword, X_test_no_keyword = divideDataOnHasKeyword(X_test)

  X_train.drop('hasKeyword', axis=1)
  X_test.drop('hasKeyword', axis = 1)
  X_test_keyword.drop('hasKeyword', axis = 1)
  X_test_no_keyword.drop('hasKeyword', axis = 1)

  if kw == 'combined':
    if kw_ == 'combined':
      rf_model_kw.fit(X_train, y_train['label'])
      gbm_model_kw.fit(X_train, y_train['label'])
      dtc_model_kw.fit(X_train, y_train['label'])
      rf_model_kw_.fit(X_train, y_train['label'])
      gbm_model_kw_.fit(X_train, y_train['label'])
      dtc_model_kw_.fit(X_train, y_train['label'])

      y_pred_rf_kw = rf_model_kw.predict(X_test_keyword)
      y_pred_gbm_kw = gbm_model_kw.predict(X_test_keyword)
      y_pred_dtc_kw = dtc_model_kw.predict(X_test_keyword)
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword)
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword)
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword)

    elif kw_ == 'keywords':
      rf_model_kw.fit(X_train, y_train['label'])
      gbm_model_kw.fit(X_train, y_train['label'])
      dtc_model_kw.fit(X_train, y_train['label'])
      rf_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])
      gbm_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])
      dtc_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])

      y_pred_rf_kw = rf_model_kw.predict(X_test_keyword)
      y_pred_gbm_kw = gbm_model_kw.predict(X_test_keyword)
      y_pred_dtc_kw = dtc_model_kw.predict(X_test_keyword)
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])

    elif kw_ == 'changes':
      rf_model_kw.fit(X_train, y_train['label'])
      gbm_model_kw.fit(X_train, y_train['label'])
      dtc_model_kw.fit(X_train, y_train['label'])
      rf_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])
      gbm_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])
      dtc_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])

      y_pred_rf_kw = rf_model_kw.predict(X_test_keyword)
      y_pred_gbm_kw = gbm_model_kw.predict(X_test_keyword)
      y_pred_dtc_kw = dtc_model_kw.predict(X_test_keyword)
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
  elif kw == 'keywords':
    if kw_ == 'combined':
      rf_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      gbm_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      dtc_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      rf_model_kw_.fit(X_train, y_train['label'])
      gbm_model_kw_.fit(X_train, y_train['label'])
      dtc_model_kw_.fit(X_train, y_train['label'])

      y_pred_rf_kw = rf_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_gbm_kw = gbm_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_dtc_kw = dtc_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword)
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword)
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword)

    elif kw_ == 'keywords':
      rf_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      gbm_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      dtc_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      rf_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])
      gbm_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])
      dtc_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])

      y_pred_rf_kw = rf_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_gbm_kw = gbm_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_dtc_kw = dtc_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])

    elif kw_ == 'changes':
      rf_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      gbm_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      dtc_model_kw.fit(X_train.iloc[:, 48:68], y_train['label'])
      rf_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])
      gbm_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])
      dtc_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])

      y_pred_rf_kw = rf_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_gbm_kw = gbm_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_dtc_kw = dtc_model_kw.predict(X_test_keyword.iloc[:, 48:68])
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
  elif kw == 'changes':
    if kw_ == 'combined':
      rf_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      gbm_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      dtc_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      rf_model_kw_.fit(X_train, y_train['label'])
      gbm_model_kw_.fit(X_train, y_train['label'])
      dtc_model_kw_.fit(X_train, y_train['label'])

      y_pred_rf_kw = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_gbm_kw = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_dtc_kw = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword)
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword)
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword)

    elif kw_ == 'keywords':
      rf_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      gbm_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      dtc_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      rf_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])
      gbm_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])
      dtc_model_kw_.fit(X_train.iloc[:, 48:68], y_train['label'])

      y_pred_rf_kw = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_gbm_kw = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_dtc_kw = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 48:68])

    elif kw_ == 'changes':
      rf_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      gbm_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      dtc_model_kw.fit(X_train.iloc[:, 0:48], y_train['label'])
      rf_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])
      gbm_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])
      dtc_model_kw_.fit(X_train.iloc[:, 0:48], y_train['label'])

      y_pred_rf_kw = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_gbm_kw = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_dtc_kw = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_rf_kw_ = rf_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_gbm_kw_ = gbm_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])
      y_pred_dtc_kw_ = dtc_model_kw_.predict(X_test_no_keyword.iloc[:, 0:48])


  y_pred_rf = np.concatenate((y_pred_rf_kw,y_pred_rf_kw_), axis=0)
  y_pred_gbm = np.concatenate((y_pred_gbm_kw,y_pred_gbm_kw_), axis=0)
  y_pred_dtc = np.concatenate((y_pred_dtc_kw,y_pred_dtc_kw_),axis=0)

  y_test_keyword, y_test_no_keyword = divideDataOnHasKeyword(y_test)

  y_test.drop('hasKeyword', axis = 1)
  y_test_keyword.drop('hasKeyword', axis = 1)
  y_test_no_keyword.drop('hasKeyword', axis = 1)

  y_test_merged = np.concatenate((y_test_keyword,y_test_no_keyword), axis=0)

  y_test_merged = y_test_merged[:,0]

  training_kappa_rf=cohen_kappa_score(y_pred_rf,y_test_merged)
  training_kappa_gbm=cohen_kappa_score(y_pred_gbm,y_test_merged)
  training_kappa_dtc=cohen_kappa_score(y_pred_dtc,y_test_merged)

  print("Testing Dataset Performance")
  print("")

  print("Random Forest")
  print("Kappa")
  print(training_kappa_rf)
  print("Confusion Matrix")
  print(confusion_matrix(y_test_merged,y_pred_rf))
  print("Classification report")
  print(classification_report(y_test_merged,y_pred_rf))
  print("Accuracy")
  print(accuracy_score(y_test_merged, y_pred_rf))

  print("")

  print("GBM")
  print("Kappa")
  print(training_kappa_gbm)
  print("Confusion Matrix")
  print(confusion_matrix(y_test_merged,y_pred_gbm))
  print("Classification report")
  print(classification_report(y_test_merged,y_pred_gbm))
  print("Accuracy")
  print(accuracy_score(y_test_merged, y_pred_gbm))

  print("")

  print("Decision Tree")
  print("Kappa")
  print(training_kappa_dtc)
  print("Confusion Matrix")
  print(confusion_matrix(y_test_merged,y_pred_dtc))
  print("Classification report")
  print(classification_report(y_test_merged,y_pred_dtc))
  print("Accuracy")
  print(accuracy_score(y_test_merged, y_pred_dtc))

  print("")

In [11]:
print('keywords-changes')
modelling('keywords', 'changes')
print('combined-combined')
modelling('combined', 'combined')
print('keywords-combined')
modelling('keywords', 'combined')
#we can add the other 6 combinations too

keywords-changes
Training Dataset Performance

Random Forest
Kappa
0.5647139079537403
Average Cross Validation score :0.7251020408163263

GBM
Kappa
0.5715288972456641
Average Cross Validation score :0.7308163265306123

Decision Tree
Kappa
0.5200890521155559
Average Cross Validation score :0.6946938775510203

Testing Dataset Performance

Random Forest
Kappa
0.5349702765572499
Confusion Matrix
[[25  5  8]
 [ 4 54 12]
 [ 9 14 42]]
Classification report
              precision    recall  f1-score   support

           a       0.66      0.66      0.66        38
           c       0.74      0.77      0.76        70
           p       0.68      0.65      0.66        65

    accuracy                           0.70       173
   macro avg       0.69      0.69      0.69       173
weighted avg       0.70      0.70      0.70       173

Accuracy
0.6994219653179191

GBM
Kappa
0.5361930294906166
Confusion Matrix
[[26  8  4]
 [ 6 55  9]
 [ 9 16 40]]
Classification report
              precision    reca